In [1]:
%matplotlib notebook

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
census = "Resources/society_redistricting_2010.gpkg"
school_areas = "Resources/bdry_school_attendance_areas.gpkg"

census_block_df = gpd.read_file(census)
school_area_df = gpd.read_file(school_areas)

In [4]:
#Assign spatial reference datum for census block and school spatial dataframes
#Note: these spatial dataframes are not in Github due to their size. They have been saved in Google drive due to file size
census_block_df = census_block_df.to_crs({'init': 'epsg:26915'})
school_area_df = school_area_df.to_crs({'init': 'epsg:26915'})
#Intersect the school and census spatial data frames
census_school_intersect = gpd.sjoin(census_block_df, school_area_df, how="inner", op='intersects')


In [6]:
#list the columns in the the intersected file
list(census_school_intersect.columns) 
#Create a subset of the intersected spatial dataframe 
census_subset=census_school_intersect[['STATE', 'BLOCKGROUP', 'BLOCK', 'TRACT', 'MCD', 'SUBMCD', 'PLACE',  'UNI_NAM', 'HIGH_NAME', 'MIDD_NAME', 'ELEM_NAME', 'POPULATION']]

In [7]:
#Create 
census_subset.to_csv('Resources/census_subset.csv')